In [ ]:
BIT_WIDTH = 8
EPOCHS = 80
Q_EPOCHS = 25
BATCH_SIZE = 256
MODEL_NAME = 'model'
MINI = False
FIT_MODEL = True
MODEL_STRUCTURE = 2 # 0: short, 1: medium, 2: long
mini = "_mini" if MINI else ""

In [ ]:
import MNIST_database as mnist
from matplotlib import pyplot as plt


#Choose the final size of your image dataset
size_final = 8

# data_zoom = mnist.MNISTData(size_initial=20, size_final=size_final, color_depth=5, flat=True)
data_zoom = mnist.MNISTData(size_initial=20, size_final=8, color_depth=8, flat=True)
# todo: data_zoom.x_train = data_zoom.x_train*(2**(BIT_WIDTH))
# todo: data_zoom.x_test = data_zoom.x_test*(2**(BIT_WIDTH))

# x_train= data_zoom.x_train
# y_train= data_zoom.y_train
# x_test= data_zoom.x_test
# y_test= data_zoom.y_test

ax = plt.subplot(1, 1 , 1)
plt.imshow(data_zoom.x_train[0].reshape(size_final,size_final), cmap='gray_r')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

# def model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from callbacks import all_callbacks
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
import tensorflow_model_optimization as tfmot
import os 
# import MNIST_database as mnist
from matplotlib import pyplot as plt

class QAutoencoder:
    def __init__(self, data: mnist.MNISTData, bit_width=8, EPOCHS=1, Q_EPOCHS=1, model_name='model'):
        self.x_train = data.x_train
        self.y_train = data.y_train
        self.x_test = data.x_test
        self.y_test = data.y_test
        self.input_min = np.min(data.x_train)
        self.input_max = np.max(data.x_train)
        self.input_shape = (data.x_train.shape[-1],)
        self.BIT_WIDTH = bit_width
        self.EPOCHS = EPOCHS
        self.Q_EPOCHS = Q_EPOCHS
        self.MODEL_NAME = model_name
        self.model = self.autoencoder_model_gen()
        self.history = None
        self.loss = None
        self.float_model_predictions = None
        self.path_to_model =    None

        self.path_to_quantized_model = None
        self.q_aware_model = None
        self.q_aware_loss = None
        self.quantized_tflite_model = None
        self.interpreter = None
        self.input_details = None
        self.output_details = None
        self.quantized_model_predictions = None
    
    def mini_model(self):
        encoder_input = Input(shape=self.input_shape)
        decoder_output = Dense(self.x_train.shape[-1], activation='linear')(encoder_input)
        # Model
        model = Model(inputs=encoder_input, outputs=decoder_output)
        # refactor the code above to use the functional AP

        model.compile(optimizer='adam', loss='mse')
        # model.compile(optimizer='adam', loss='binary_crossentropy') # classifier
        return model
    
    def autoencoder_model_gen(self):
        if MINI:
            return self.mini_model()
        # Encoder
        # encoder_input = Input(shape=self.input_shape)
        # encoder_l1 = Dense(64, activation='relu')(encoder_input)
        # encoder_l2 = Dense(32, activation='relu')(encoder_l1)
        # encoder_l3 = Dense(16, activation='relu')(encoder_l2)
        # encoder_output = Dense(2, activation='relu')(encoder_l3)
        
        # # Decoder
        # decoder_l1 = Dense(16, activation='relu')(encoder_output)
        # decoder_l2 = Dense(32, activation='relu')(decoder_l1)
        # decoder_l3 = Dense(32, activation='relu')(decoder_l2)
        if MODEL_STRUCTURE == 0:
            # Encoder
            encoder_input = Input(shape=self.input_shape)
            encoder_l1 = Dense(64, activation='relu')(encoder_input)
            encoder_l3 = Dense(15, activation='relu')(encoder_l2)
            encoder_output = Dense(2, activation='relu')(encoder_l4)

            # Decoder
            # decoder_l1 = Dense(16, activation='relu')(encoder_output)
            decoder_l2 = Dense(32, activation='relu')(encoder_output)
            # decoder_output = Dense(y_train.shape[-1], activation='sigmoid')(decoder_l3) # classifier
            # decoder_output = Dense(
            #     self.x_train.shape[-1], activation='sigmoid')(decoder_l3)  # autoencoder
            decoder_output = Dense(self.x_train.shape[-1], activation='linear')(decoder_l3)

        if MODEL_STRUCTURE == 1:
            # Encoder
            encoder_input = Input(shape=self.input_shape)
            encoder_l1 = Dense(64, activation='relu')(encoder_input)
            encoder_l2 = Dense(15, activation='relu')(encoder_l1)
            encoder_l3 = Dense(15, activation='relu')(encoder_l2)
            encoder_l4 = Dense(15, activation='relu')(encoder_l3)
            encoder_output = Dense(2, activation='relu')(encoder_l4)

            # Decoder
            # decoder_l1 = Dense(16, activation='relu')(encoder_output)
            decoder_l2 = Dense(32, activation='relu')(encoder_output)
            decoder_l3 = Dense(32, activation='relu')(decoder_l2)
            # decoder_output = Dense(y_train.shape[-1], activation='sigmoid')(decoder_l3) # classifier
            # decoder_output = Dense(
            #     self.x_train.shape[-1], activation='sigmoid')(decoder_l3)  # autoencoder
            decoder_output = Dense(self.x_train.shape[-1], activation='linear')(decoder_l3)

        if MODEL_STRUCTURE == 2:
            # Encoder
            encoder_input = Input(shape=self.input_shape)
            encoder_l1 = Dense(64, activation='relu')(encoder_input)
            encoder_l2 = Dense(50, activation='relu')(encoder_l1)
            encoder_l3 = Dense(32, activation='relu')(encoder_l2)
            encoder_l4 = Dense(16, activation='relu')(encoder_l3)
            encoder_l5 = Dense(8, activation='relu')(encoder_l4)
            encoder_l6 = Dense(4, activation='relu')(encoder_l5)
            encoder_output = Dense(2, activation='relu')(encoder_l6)

            # Decoder
            decoder_l1 = Dense(16, activation='relu')(encoder_output)
            decoder_l2 = Dense(32, activation='relu')(decoder_l1)
            decoder_l3 = Dense(32, activation='relu')(decoder_l2)
            # decoder_output = Dense(y_train.shape[-1], activation='sigmoid')(decoder_l3) # classifier
            # decoder_output = Dense(
            #     self.x_train.shape[-1], activation='sigmoid')(decoder_l3)  # autoencoder
            decoder_output = Dense(self.x_train.shape[-1], activation='linear')(decoder_l3)

        # Model
        model = Model(inputs=encoder_input, outputs=decoder_output)
        # refactor the code above to use the functional AP

        model.compile(optimizer='adam', loss='mse')
        # model.compile(optimizer='adam', loss='binary_crossentropy') # classifier
        return model

    def fit_data(self, batch_size=256, epochs=EPOCHS):
        """Write the fit function for the autoencoder. 
        Storing the fit history in self.history to be able to plot the fitting scores."""

        callbacks = all_callbacks(stop_patience=1000,
                                  lr_factor=0.5,
                                  lr_patience=10,
                                  lr_epsilon=0.000001,
                                  # min_delta=0.000001,
                                  lr_cooldown=2,
                                  lr_minimum=0.0000001,
                                outputDir=f'model{mini}/callbacks')

        self.history = self.model.fit(self.x_train, self.x_train,
                                      validation_data=(
                                          self.x_test, self.x_test),
                                      batch_size=batch_size, epochs=epochs,
                                      shuffle=True, callbacks=callbacks.callbacks)
        # self.model = strip_pruning(self.model)
        self.loss = self.model.evaluate(self.x_test, self.x_test, verbose=0)
        self.path_to_model =    f'model{mini}/model_{self.loss}loss/KERAS_check_best_model{mini}.model'
        if not os.path.exists(self.path_to_model):
            os.makedirs(self.path_to_model)

        self.model.save(
           self.path_to_model)
        self.history = self.history.history
        self.convert_to_Q_aware()

    def plot_float_model(self, n=6):
        """Plot the float model"""
        
        plt.figure(figsize=(10, 3))
        self.float_model_predictions = self.model.predict(self.x_test)
        self.plot_model_predictions(
            n,
            self.float_model_predictions,
            f'./images/float_model/reconstructed images {self.MODEL_NAME}{mini}.png',
        )

    def representative_dataset(self):
        for data in self.x_train:
            # Scale the data using min and max values
            scaled_data = (data - self.input_min) / (self.input_max - self.input_min) * (2 ** (self.BIT_WIDTH - 1))
            yield [np.array([scaled_data], dtype=np.float32)]

    def convert_to_Q_aware(self):
        quantize_model = tfmot.quantization.keras.quantize_model
        self.q_aware_model = quantize_model(self.model)
        self.q_aware_model.compile(optimizer='adam', loss='mse')
        self.fit_data_Q_aware()
        # loss = self.q_aware_model.evaluate(self.x_test, self.x_test, verbose=0)

        # # Save the quantized model
        # with open(f'quantized_model.tflite', 'wb') as f:
        #     f.write(self.q_aware_model)

        self.convert_to_tflite()

    def fit_data_Q_aware(self, batch_size=256, epochs=Q_EPOCHS):
        """Write the fit function for the autoencoder. 
        Storing the fit history in self.history to be able to plot the fitting scores."""
        
        self.path_to_quantized_model = f'model{mini}/model_{self.loss}loss/QAE_model{self.BIT_WIDTH}bits/KERAS_check_best_model{mini}.model'
        
        if not os.path.exists(self.path_to_quantized_model):
            os.makedirs(self.path_to_quantized_model)

        callbacks = all_callbacks(stop_patience=1000,
                                  lr_factor=0.5,
                                  lr_patience=10,
                                  lr_epsilon=0.000001,
                                  # min_delta=0.000001,
                                  lr_cooldown=2,
                                  lr_minimum=0.0000001,
                                  outputDir=f'model{mini}/model_{self.loss}loss/QAE_model{self.BIT_WIDTH}bits/callbacks')
        # callbacks.callbacks.append(pruning_callbacks.UpdatePruningStep())

        self.history_Q_aware = self.q_aware_model.fit(self.x_train, self.x_train,
                                      validation_data=(
                                          self.x_test, self.x_test),
                                      batch_size=batch_size, epochs=epochs,
                                      shuffle=True, callbacks=callbacks.callbacks)
        # self.model = strip_pruning(self.model)

        self.q_aware_model.save(
            self.path_to_quantized_model)
        self.history_Q_aware = self.history_Q_aware.history
        self.q_aware_loss = self.q_aware_model.evaluate(self.x_test, self.x_test, verbose=0)
        # self.convert_to_Q_aware()    

    def convert_to_tflite(self):
        converter = tf.lite.TFLiteConverter.from_keras_model(self.q_aware_model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = self.representative_dataset
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.int8
        converter.inference_output_type = tf.int8

        self.quantized_tflite_model = converter.convert()

        # Save the TensorFlow Lite model
        path_to_tflite_model = f'model{mini}/model_{self.loss}loss/model_obj_tflite_model{mini}'
        if not os.path.exists(path_to_tflite_model):
            os.makedirs(path_to_tflite_model)
        with open(f"{path_to_tflite_model}.tflite", "wb") as file:
            file.write(self.quantized_tflite_model)

        # Load the quantized model
        self.interpreter = tf.lite.Interpreter(model_content=self.quantized_tflite_model)

        self.interpreter.allocate_tensors()

        # Get input and output details
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()
        self.quantized_predictions()

    def quantized_predictions(self, n=50):
        quantized_model_predictions = []

        for i in range(n):
            # Prepare input data
            # input_data = np.array(
            #     [self.x_test[i]*(2**(self.BIT_WIDTH-1))], dtype=np.int8)
            # input_data = np.array([(self.x_test[i] - self.input_min) / (self.input_max - self.input_min) * (2 ** (self.BIT_WIDTH - 1))], dtype=np.int8)  
            input_data = np.array([(self.x_test[i] - self.input_min) / (self.input_max - self.input_min) * (2 ** (self.BIT_WIDTH))], dtype=np.int8)  

            self.interpreter.set_tensor(
                self.input_details[0]['index'], input_data)
            # print(f"input_data: {input_data}")

            # Run inference
            self.interpreter.invoke()

            # Get output
            output_data = self.interpreter.get_tensor(self.output_details[0]['index'])
            output_data = self.interpreter.get_tensor(
                # self.output_details[0]['index']) / (2 ** (self.BIT_WIDTH - 1))
                self.output_details[0]['index'])
            # output_data = output_data * (self.input_max - self.input_min) / (2 ** (self.BIT_WIDTH - 1)) + self.input_min
            # output_data = output_data / (2 ** (self.BIT_WIDTH - 1))
            quantized_model_predictions.append(output_data)
            # print(f"output_data: {output_data}")
        
        self.quantized_model_predictions = quantized_model_predictions
        # self.compute_mse()

    def quantized_predictions2(self, n=50):
        quantized_model_predictions = []

        for i in range(n):
            # Prepare input data
            # input_data = np.array(
            #     [self.x_test[i]*(2**(self.BIT_WIDTH-1))], dtype=np.int8)
            # input_data = np.array([(self.x_test[i] - self.input_min) / (self.input_max - self.input_min) * (2 ** (self.BIT_WIDTH - 1))], dtype=np.int8)  
            input_data = np.array([(self.x_test[i] - self.input_min) / (self.input_max - self.input_min) * (2 ** (self.BIT_WIDTH))], dtype=np.int64)  

            self.interpreter.set_tensor(
                self.input_details[0]['index'], input_data)
            # print(f"input_data: {input_data}")

            # Run inference
            self.interpreter.invoke()

            # Get output
            output_data = self.interpreter.get_tensor(self.output_details[0]['index'])
            output_data = self.interpreter.get_tensor(
                # self.output_details[0]['index']) / (2 ** (self.BIT_WIDTH - 1))
                self.output_details[0]['index'])
            # output_data = output_data * (self.input_max - self.input_min) / (2 ** (self.BIT_WIDTH - 1)) + self.input_min
            # output_data = output_data / (2 ** (self.BIT_WIDTH - 1))
            quantized_model_predictions.append(output_data)
            # print(f"output_data: {output_data}")
        
        self.quantized_model_predictions = quantized_model_predictions
        # self.compute_mse()

    def plot_quantized_model(self, n=6):

        
        plt.figure(figsize=(10, 3))
        self.plot_model_predictions(
            n,
            self.quantized_model_predictions,
            f'./images/QAE/reconstructed images{self.MODEL_NAME}{mini}.png',
        )

    def plot_model_predictions(self, n, quantized_model_predictions, imgs_path):
        img_size = int(np.sqrt(self.input_shape[0]))
        for i in range(n):
            ax = plt.subplot(2, n, i + 1)
            self.plot_imgs(
                self.x_test, i, img_size, ax
            )
            ax = plt.subplot(2, n, i + n + 1)
            self.plot_imgs(
                quantized_model_predictions, i, img_size, ax
            )
        # if not os.path.exists(imgs_path):
        #     os.makedirs(imgs_path)
        plt.savefig(imgs_path.format(model_name=" complete"))
        plt.show()

    def plot_imgs(self, arg0, i, img_size, ax):
        plt.imshow(arg0[i].reshape(img_size, img_size), cmap='gray_r')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    def compute_mse(self):
        mse = mean_squared_error(self.float_model_predictions, self.quantized_model_predictions)
        print(f'Mean Squared Error between floating point and quantized model predictions: {mse}')
        self.mse = mse

model_obj = QAutoencoder(data_zoom, bit_width=BIT_WIDTH, model_name=MODEL_NAME, EPOCHS=EPOCHS, Q_EPOCHS=Q_EPOCHS)
# model.fit(x_train, y_train, epochs=10, batch_size=32) # classifier
model_obj.model.summary()
# model.fit(x_train, x_train, epochs=EPOCHS, batch_size=BATCH_SIZE) #autoencoder


# fit

In [ ]:
if FIT_MODEL:
    model_obj.fit_data(epochs=EPOCHS)  # batch_size=BATCH_SIZE, epochs=EPOCHS)


# Save objects

In [ ]:
if FIT_MODEL:
    print(model_obj.loss)

In [ ]:
# model_obj.plot_float_model()
# model_obj.plot_quantized_model(n=6)

## saving as pickle

In [ ]:
# import os

# MODEL_DIR = os.path.abspath(f"model{mini}")
# # ASSETS_DIR = os.path.join(MODEL_DIR, "assets")

# # if not os.path.exists(MODEL_DIR):
# #     os.makedirs(MODEL_DIR)

# # if not os.path.exists(ASSETS_DIR):
# #     os.makedirs(ASSETS_DIR)
# MODEL_DIR

In [ ]:
# model_obj.path_to_model.split('/')[:-1]
# join model path
'/'.join(model_obj.path_to_model.split('/')[:-1])

In [ ]:
os.listdir('/'.join(model_obj.path_to_model.split('/')[:-1]))

In [ ]:
import pickle


def save_objects(model_obj):
    save_path = '/'.join(model_obj.path_to_model.split('/')[:-1])
    # # Save the Keras model
    # model_obj.model.save(f'{model_obj.path_to_model}')

    # # Save the .tflite model
    # with open(f"{model_obj.path_to_model}/model_obj_tflite_model{mini}.tflite", "wb") as file:
    #     file.write(model_obj.quantized_tflite_model)
    # Save other attributes
    attrs_to_save = {
        'x_train': model_obj.x_train,
        # 'y_train': model_obj.y_train,
        'x_test': model_obj.x_test,
        # 'y_test': model_obj.y_test,
        'input_min': model_obj.input_min,
        'input_max': model_obj.input_max,
        'input_shape': model_obj.input_shape,
        'BIT_WIDTH': model_obj.BIT_WIDTH,
        'EPOCHS': model_obj.EPOCHS,
        'Q_EPOCHS': model_obj.Q_EPOCHS,
        'MODEL_NAME': model_obj.MODEL_NAME,
        'history': model_obj.history,
        'loss': model_obj.loss,
        'float_model_predictions': model_obj.float_model_predictions,
        'path_to_model': model_obj.path_to_model,

        'path_to_quantized_model': model_obj.path_to_quantized_model,
        # 'q_aware_model': model_obj.q_aware_model,
        'q_aware_loss': model_obj.q_aware_loss,
        'quantized_tflite_model': model_obj.quantized_tflite_model,
        # 'interpreter': model_obj.interpreter,
        # 'input_details': model_obj.input_details,
        # 'output_details': model_obj.output_details,
        'quantized_model_predictions': model_obj.quantized_model_predictions
        # 'history_Q_aware': model_obj.history_Q_aware,
        # 'mse': model_obj.mse,
    }
    with open(f'{save_path}/model_obj_attributes{mini}.pickle', 'wb') as f:
        pickle.dump(attrs_to_save, f)


# Save the objects of model_obj
if FIT_MODEL:
    save_objects(model_obj)        

# Load objects

## load it 

### definitions

In [ ]:
# BIT_WIDTH = 8
# EPOCHS = 80
# Q_EPOCHS = 40
# BATCH_SIZE = 256
# MODEL_NAME = 'model'

In [ ]:
# list all folders in the given directory
import os

def get_best_model_path(path):
    if directories := [
    d
    for d in os.listdir(path)
    if d.startswith('model_') and d.endswith('loss')
]:
    # Extract loss value from each directory name
        losses = [float(d.split('_')[1].replace('loss', '')) for d in directories]

    # Select directory with lowest loss value
        lowest_loss_index = losses.index(min(losses))
        lowest_loss_dir = directories[lowest_loss_index]
        print(f"The directory with the lowest loss is {lowest_loss_dir}")
        return f"{path}/{lowest_loss_dir}"

    else:
        print("No directories found with name pattern 'model_*loss'")
        return ""


path = f'./model{mini}'  # Replace with the path to the directory you want to list    
MODEL_DIR = get_best_model_path(path)

MODEL_DIR

In [ ]:
f"{MODEL_DIR}/KERAS_check_best_model{mini}.model"

In [ ]:
import pickle

def load_objects():
    
    # model_obj.model.save('model_obj_keras_model.h5')
    # Load the TensorFlow model
    loaded_model = tf.keras.models.load_model(f"{MODEL_DIR}/KERAS_check_best_model{mini}.model")

    # Load the Keras model
    # loaded_keras_model = tf.keras.models.load_model(f'{MODEL_DIR}/model_obj_keras_model{mini}.h5')

    # Load the .tflite model
    with open(f"{MODEL_DIR}/model_obj_tflite_model{mini}.tflite", "rb") as file:
        loaded_tflite_model = file.read()

    # Load other attributes
    with open(f'{MODEL_DIR}/model_obj_attributes{mini}.pickle', 'rb') as f:
        loaded_attributes = pickle.load(f)
    
    # Create a new QAutoencoder object and set the attributes
    loaded_model_obj = QAutoencoder(data_zoom, bit_width=loaded_attributes['BIT_WIDTH'], model_name=loaded_attributes['MODEL_NAME'],
                                    EPOCHS=loaded_attributes['EPOCHS'], Q_EPOCHS=loaded_attributes['Q_EPOCHS'])
    
    loaded_model_obj.model = loaded_model
    # loaded_model_obj.model = loaded_keras_model
    loaded_model_obj.quantized_tflite_model = loaded_tflite_model

    loaded_model_obj.x_train = loaded_attributes['x_train']
    # loaded_model_obj.y_train = loaded_attributes['y_train']
    loaded_model_obj.x_test = loaded_attributes['x_test']
    # loaded_model_obj.y_test = loaded_attributes['y_test']
    loaded_model_obj.input_min = loaded_attributes['input_min']
    loaded_model_obj.input_max = loaded_attributes['input_max']
    loaded_model_obj.input_shape = loaded_attributes['input_shape']
    loaded_model_obj.BIT_WIDTH = loaded_attributes['BIT_WIDTH']
    loaded_model_obj.EPOCHS = loaded_attributes['EPOCHS']
    loaded_model_obj.Q_EPOCHS = loaded_attributes['Q_EPOCHS']
    loaded_model_obj.MODEL_NAME = loaded_attributes['MODEL_NAME']
    loaded_model_obj.history = loaded_attributes['history']
    loaded_model_obj.loss = loaded_attributes['loss']
    loaded_model_obj.float_model_predictions = loaded_attributes['float_model_predictions']
    loaded_model_obj.path_to_model = loaded_attributes['path_to_model']

    loaded_model_obj.path_to_quantized_model = loaded_attributes['path_to_quantized_model']
    # loaded_model_obj.q_aware_model = loaded_attributes['q_aware_model']
    loaded_model_obj.q_aware_loss = loaded_attributes['q_aware_loss']
    loaded_model_obj.quantized_tflite_model = loaded_attributes['quantized_tflite_model']
    # loaded_model_obj.interpreter = loaded_attributes['interpreter']
    # loaded_model_obj.input_details = loaded_attributes['input_details']
    # loaded_model_obj.output_details = loaded_attributes['output_details']
    loaded_model_obj.quantized_model_predictions = loaded_attributes['quantized_model_predictions']
    
    # loaded_model_obj.interpreter = loaded_tflite_model
    # loaded_model_obj.mse = loaded_attributes['mse']

    # Recreate the TFLite interpreter
    loaded_model_obj.interpreter = tf.lite.Interpreter(model_content=loaded_tflite_model)
    loaded_model_obj.interpreter.allocate_tensors()
    loaded_model_obj.input_details = loaded_model_obj.interpreter.get_input_details()
    loaded_model_obj.output_details = loaded_model_obj.interpreter.get_output_details()
    # loaded_model_obj.convert_to_Q_aware()

    return loaded_model_obj

# # Load the objects and create a new model_obj
# loaded_model_obj = load_objects()

### loaded model

In [ ]:
# Load the objects and create a new model_obj
loaded_model_obj = load_objects()

In [ ]:
loaded_model_obj.loss

In [ ]:
loaded_model_obj.x_train.shape

In [ ]:
loaded_model_obj.model.summary()

In [ ]:
loaded_model_obj.plot_float_model()


In [ ]:
loaded_model_obj.quantized_predictions(n=20)
loaded_model_obj.plot_quantized_model(n=20)

In [ ]:
# loaded_model_obj.quantized_predictions2(n=20)
# loaded_model_obj.plot_quantized_model(n=20)

In [ ]:
# loaded_model_obj.quantized_model_predictions

In [ ]:
# for i, img in enumerate(loaded_model_obj.quantized_model_predictions):
#     print(f' ------------ Image {i} ------------')
#     for j, row in enumerate(img):
#         for k, pixel in enumerate(row):
#             print(pixel, " : ", Fxp(pixel, signed=is_signed, n_word=BIT_WIDTH, n_frac=0).bin())
#             # print(Fxp(pixel, signed=is_signed, n_word=BIT_WIDTH, n_frac=0).bin())
#             print(' ')
        

In [ ]:
from fxpmath import Fxp
# BIT_WIDTH = 8
is_signed = True
TB_FILES_DIR = 'model/testbench_files'
if not os.path.exists(TB_FILES_DIR):
    os.makedirs(TB_FILES_DIR)

In [ ]:
quantized_model_predictions = []
pred_len = len(loaded_model_obj.quantized_model_predictions)

with open('model/testbench_files/inputs_string.txt', 'w') as f:

    for img in range(pred_len):
        # Prepare input data
        input_data = np.array([(loaded_model_obj.x_test[img] - loaded_model_obj.input_min) / (loaded_model_obj.input_max - loaded_model_obj.input_min) * (2 ** (loaded_model_obj.BIT_WIDTH))], dtype=np.int8)

        for pixel in input_data[0]:
            fxp_item = Fxp(pixel, signed=True, n_word=loaded_model_obj.BIT_WIDTH, n_frac=0)
            # print(pixel, " = ", fxp_item, " : ", fxp_item.bin())

            f.write(fxp_item.bin())
        f.write("\n")

        # print(f"input_data: {input_data}")



In [ ]:
#  generating expected results for testbench
with open(f'{TB_FILES_DIR}/expected_results.txt', 'w') as f:
    for img in loaded_model_obj.quantized_model_predictions:
        for row in img:
            for pixel in row:
                # print(pixel, " : ", Fxp(pixel, signed=is_signed, n_word=BIT_WIDTH, n_frac=0).bin())
                f.write(Fxp(pixel, signed=is_signed, n_word=loaded_model_obj.BIT_WIDTH, n_frac=0).bin())
        # print(" ")
        f.write('\n')

In [ ]:
#  generating expected results for testbench
with open(f'{TB_FILES_DIR}/expected_results_dec.txt', 'w') as f:
    for img in loaded_model_obj.quantized_model_predictions:
        for row in img:
            for pixel in row:
                # print(pixel, " : ", Fxp(pixel, signed=is_signed, n_word=BIT_WIDTH, n_frac=0).bin())
                f.write(f"{str(pixel)} ")
        # print(" ")
        f.write('\n')

In [ ]:
# loaded_model_obj.model.get_weights()

In [ ]:
# loaded_model_obj.quantized_tflite_model

# Clone and fine-tune pre-trained model with quantization aware training


In [ ]:
model_obj.model

In [ ]:
# import tensorflow_model_optimization as tfmot
# model = model_obj.model
# quantize_model = tfmot.quantization.keras.quantize_model

# # q_aware stands for for quantization aware.
# q_aware_model = quantize_model(model)

# # `quantize_model` requires a recompile.
# # q_aware_model.compile(optimizer='adam',
# #               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
# #               metrics=['accuracy'])
# q_aware_model.compile(optimizer='adam', loss='mse')

# q_aware_model.summary()

### Train and evaluate the model against baseline

To demonstrate fine tuning after training the model for just an epoch, fine tune with quantization aware training on a subset of the training data.

In [ ]:
model_obj.x_train[0:1000]

In [ ]:
# train_images_subset = model_obj.x_train[0:1000] # out of 60000
# train_labels_subset = model_obj.x_train[0:1000]

# q_aware_model.fit(train_images_subset, train_labels_subset,
#                   batch_size=500, epochs=1)

In [ ]:
# baseline_model_accuracy = model_obj.model.evaluate(
#     model_obj.x_test, model_obj.x_test, verbose=0)

# q_aware_model_accuracy = model_obj.q_aware_model.evaluate(
#    model_obj.x_test, model_obj.x_test, verbose=0)

# print('Baseline test accuracy:', baseline_model_accuracy)
# print('Quant test accuracy:', q_aware_model_accuracy)

## Create quantized model for TFLite backend

After this, you have an actually quantized model with int8 weights and uint8 activations.

In [ ]:
# converter = tf.lite.TFLiteConverter.from_keras_model(model_obj.q_aware_model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]

# q_aware_model = converter.convert()

## See persistence of accuracy from TF to TFLite

Define a helper function to evaluate the TF Lite model on the test dataset.

In [ ]:
# import numpy as np

# def evaluate_model(interpreter, x_test, n=6):
# #   input_index = interpreter.get_input_details()[0]["index"]
# #   output_index = interpreter.get_output_details()[0]["index"]
#     input_details = interpreter.get_input_details()
#     output_details = interpreter.get_output_details()

# #   # Run predictions on every image in the "test" dataset.
# #   prediction_digits = []
# #   for i, test_image in enumerate(test_images):
# #     if i % 1000 == 0:
# #       print('Evaluated on {n} results so far.'.format(n=i))
# #     # Pre-processing: add batch dimension and convert to float32 to match with
# #     # the model's input data format.
# #     test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
# #     interpreter.set_tensor(input_index, test_image)

# #     # Run inference.
# #     interpreter.invoke()

# #     # Post-processing: remove batch dimension and find the digit with highest
# #     # probability.
# #     output = interpreter.tensor(output_index)
# #     digit = np.argmax(output()[0])
# #     prediction_digits.append(digit)

# #   print('\n')
# #   # Compare prediction results with ground truth labels to calculate accuracy.
# #   prediction_digits = np.array(prediction_digits)
# #   accuracy = (prediction_digits == test_labels).mean()
# #   return accuracy

#     quantized_model_predictions = []

#     for i in range(n):
#         # Prepare input data
#         # input_data = np.array(
#         #     [x_test[i]*(2**(BIT_WIDTH-1))], dtype=np.int8)
#         # input_data = np.array([(x_test[i] - input_min) / (input_max - input_min) * (2 ** (BIT_WIDTH - 1))], dtype=np.int8)  
#         input_data = np.expand_dims(x_test[i], axis=0).astype(np.float32)


#         interpreter.set_tensor(
#             input_details[0]['index'], input_data)
#         # print(f"input_data: {input_data}")
#         # Run inference
#         interpreter.invoke()

#         # Get output
#         output_data = interpreter.get_tensor(output_details[0]['index'])
#         # output_data = interpreter.get_tensor(
#         #     output_details[0]['index']) / (2 ** (BIT_WIDTH - 1))
#         # output_data = output_data * (input_max - input_min) / (2 ** (BIT_WIDTH - 1)) + input_min
#         # output_data = output_data / (2 ** (BIT_WIDTH - 1))
#         # print(f"output_data: {output_data}")
#         quantized_model_predictions.append(output_data)
#     accuracy = (input_data == quantized_model_predictions).mean()
    
#     # quantized_model_predictions = quantized_model_predictions
#     return accuracy

You evaluate the quantized model and see that the accuracy from TensorFlow persists to the TFLite backend.

In [ ]:
# interpreter = tf.lite.Interpreter(model_content=q_aware_model)
# interpreter.allocate_tensors()

# # test_accuracy = evaluate_model(interpreter, test_images=model_obj.x_test, test_labels=model_obj.x_test)
# test_accuracy = evaluate_model(interpreter, model_obj.x_test, n=6)

# print('Quant TF test accuracy:', q_aware_model_accuracy)
# print('Quant TFLite test_accuracy:', test_accuracy)

In [ ]:
# model_obj.q_aware_model.layers

In [ ]:
# for layer in model_obj.q_aware_model.layers:
#   if hasattr(layer, 'quantize_config'):
#     for weight, quantizer, quantizer_vars in layer._weight_vars:
#         quantized_and_dequantized = quantizer(weight, training=False, weights=quantizer_vars)
#         min_var = quantizer_vars['min_var']
#         max_var = quantizer_vars['max_var']
#         print(quantized_and_dequantized*(2**(BIT_WIDTH)))
#         # quantized = dequantize(quantized_and_dequantized, min_var, max_var, quantizer)

In [ ]:
# model_obj.quantized_tflite_model.layers

In [ ]:
# model_obj.interpreter.layers

In [ ]:
# model_obj.plot_float_model()

In [ ]:
# # obj = model_obj
# # plot_quantized_model(obj, n=6)
# model_obj.plot_quantized_model(n=6)

## See 4x smaller model from quantization

You create a float TFLite model and then see that the quantized TFLite model
is 4x smaller.

In [ ]:
import tempfile
import os

In [ ]:
# # Create float TFLite model.
# float_converter = tf.lite.TFLiteConverter.from_keras_model(model_obj.model)
# float_tflite_model = float_converter.convert()

# # Measure sizes of models.
# _, float_file = tempfile.mkstemp('.tflite')
# _, quant_file = tempfile.mkstemp('.tflite')

# with open(quant_file, 'wb') as f:
#   f.write(model_obj.quantized_tflite_model)

# with open(float_file, 'wb') as f:
#   f.write(float_tflite_model)

# print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
# print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))
# print(float_converter.get_tensor_details())